In [4]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [2]:
mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils

In [5]:
#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))

In [6]:
# extract keypoints of the features in image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

In [5]:
os.path.join("Sign_Images")

'Sign_Images'

In [7]:
#path for flatten datas
dataPath = os.path.join('Sign_Data')

#path for images
imgPath= os.path.join('Sign_Image')

signs=np.array(["Hello","Food","Washroom"])

#30 video of each sign
numSequences=30

#length of each video
sequenceLength=30

In [8]:
for sign in signs:
    for sequence in range(numSequences):
            try:
                os.makedirs(os.path.join(dataPath,sign,str(sequence)))
                os.makedirs(os.path.join(imgPath,sign,str(sequence)))
            except:
                pass
# if not os.path.exists("Sign_Data"):
#     os.mkdir("Sign_Data")
# else:
#     print("already exist")


In [8]:
os.path??

In [9]:
# while True:
#     inpt=input(print("Select which data are you collecting:\n0: Hello \n1: Thank you"))
#     if inpt == '0' or inpt == '1':
#         choice=signs[int(inpt)]
#         break
#     else:
#         print("Invalid input, please select again!!")
# choice   

In [10]:
vid=cv2.VideoCapture(0)
with mpHolistic.Holistic() as holistic:

    #taking label input 
     while True:
        blankImg = np.zeros(shape=[512, 512, 3], dtype=np.uint8)
        cv2.putText(blankImg,"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"0: Hello , 1: Thank You", (10,100),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"2: Hungry , 3: Food", (10,150),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"4: Hospital , 5: Washroom", (10,200),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"'ESC' to escape", (10,250),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('Select label',blankImg)
        inpt=cv2.waitKey(0)
        if inpt == 48 or inpt == 49 or inpt == 50 or inpt == 51 or inpt == 52 or inpt == 53:
            choice=signs[inpt-48]
            cv2.destroyWindow('Select label')
            break
        else:
            if inpt==27:
                break
            
    #checking camera is opened or not and taking data    
     while vid.isOpened() and inpt!=27 :
        for sequence in range(numSequences):
            for frameNum in range(sequenceLength+1):
                #checks for user input to close the windows                
                key=cv2.waitKey(1)                  
                success,img=vid.read()
                
                #checking if data is accessed or not from camera
                if not success:
                    break  
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                # img.flags.writeable=False
                results=holistic.process(img) 
                # img.flags.writeable=True     
                img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
                
                #draws landmarks
                drawLandmarks()
                
                #show feed for collecting datas and delays for 2 sec
                if frameNum == 0: 
                    cv2.putText(img, "Press 'ESC' to escape", (10,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.putText(img, 'press any key to conitnue', (10,60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 0), 1, cv2.LINE_AA)
                    cv2.imshow('Collecting Datas', img)
                    key= cv2.waitKey(0)
                    cv2.waitKey(1000)
                    
                #starts collecting datas    
                else: 
                    cv2.putText(img, f"Collecting Data for '{choice}' Video Number {sequence}", (15,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.imshow('Collecting Datas', img)
                    keypoints=extractKeypoints()  
                    
                    #saving flatten array to datapath
                    np.save(os.path.join(dataPath,choice,str(sequence),str(frameNum-1)),keypoints)
                    
                    #saving image to image path
                    jpgPath=os.path.join(imgPath,choice,str(sequence),str(frameNum-1))
                    cv2.imwrite(f"{jpgPath}.jpg",img)
                    key=2 #giving default value for key to avoid esc while taking data
                
                if key == 27 : #press esc to close the window
                    break     

            if key == 27 : #press esc to close the window
                break        
        if key == 27 : #press esc to close the window
            break             
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()            

# Preprocess Data and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(signs)}

In [11]:
label_map

{'Hello': 0, 'Food': 1, 'Washroom': 2}

In [12]:
sequences, labels = [], []
for sign in signs:
    for sequence in range(numSequences):
        window = []
        for frameNum in range(sequenceLength):
            res = np.load(os.path.join(dataPath, sign, str(sequence), "{}.npy".format(frameNum)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[sign])

In [13]:
np.array(labels).shape

(90,)

In [14]:
np.array(sequences).shape

(90, 30, 1662)

In [15]:
arr=np.array([[[1,2],[3,4],[5,6]],[[1,2],[3,4],[5,6]]])

In [16]:
arr.shape

(2, 3, 2)

In [17]:
X = np.array(sequences)

In [18]:
X.shape

(90, 30, 1662)

In [19]:
y = to_categorical(labels).astype(int)

In [20]:
y.shape

(90, 3)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [21]:
y_test.shape

(9, 3)

# Save test datas and signs

In [22]:
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
try:
    os.makedirs(os.path.join(testDatasPath))
    os.makedirs(os.path.join(signsPath))
except:
    pass
np.save(os.path.join(testDatasPath,'x_test.npy'),X_test)
np.save(os.path.join(testDatasPath,'y_test.npy'),y_test)
np.save(os.path.join(signsPath,'signs.npy'),signs)

# Build and Train LSTM Neural Network

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [79]:
def createModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(signs.shape[0], activation='softmax'))
    return model

In [80]:
model=createModel()

In [81]:
res = [.7, 0.2, 0.1]

In [82]:
signs[np.argmax(res)]

'Hello'

In [83]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [84]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [85]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 5s 215ms/step - loss: 13.7361 - categorical_accuracy: 0.2469
Epoch 2/2000
3/3 [==============================] - 1s 198ms/step - loss: 85.0427 - categorical_accuracy: 0.3951
Epoch 3/2000
3/3 [==============================] - 1s 188ms/step - loss: 46.4979 - categorical_accuracy: 0.3457
Epoch 4/2000
3/3 [==============================] - 1s 199ms/step - loss: 57.7868 - categorical_accuracy: 0.4198
Epoch 5/2000
3/3 [==============================] - 1s 231ms/step - loss: 66.1208 - categorical_accuracy: 0.3333
Epoch 6/2000
3/3 [==============================] - 1s 229ms/step - loss: 47.1211 - categorical_accuracy: 0.3457
Epoch 7/2000
3/3 [==============================] - 1s 214ms/step - loss: 234.5405 - categorical_accuracy: 0.3704
Epoch 8/2000
3/3 [==============================] - 1s 290ms/step - loss: 114.2133 - categorical_accuracy: 0.2840
Epoch 9/2000
3/3 [==============================] - 1s 256ms/step - loss: 133.9644 - categoric

3/3 [==============================] - 1s 236ms/step - loss: 1.5918 - categorical_accuracy: 0.2716
Epoch 74/2000
3/3 [==============================] - 1s 224ms/step - loss: 1.3436 - categorical_accuracy: 0.4815
Epoch 75/2000
3/3 [==============================] - 1s 235ms/step - loss: 1.4999 - categorical_accuracy: 0.3086
Epoch 76/2000
3/3 [==============================] - 1s 207ms/step - loss: 1.3319 - categorical_accuracy: 0.4074
Epoch 77/2000
3/3 [==============================] - 1s 218ms/step - loss: 1.3753 - categorical_accuracy: 0.2469
Epoch 78/2000
3/3 [==============================] - 1s 210ms/step - loss: 1.0788 - categorical_accuracy: 0.4568
Epoch 79/2000
3/3 [==============================] - 1s 242ms/step - loss: 1.2188 - categorical_accuracy: 0.4198
Epoch 80/2000
3/3 [==============================] - 1s 221ms/step - loss: 1.2221 - categorical_accuracy: 0.3951
Epoch 81/2000
3/3 [==============================] - 1s 247ms/step - loss: 1.3451 - categorical_accuracy: 0.44

3/3 [==============================] - 1s 224ms/step - loss: 1.0370 - categorical_accuracy: 0.3210
Epoch 146/2000
3/3 [==============================] - 1s 250ms/step - loss: 1.0572 - categorical_accuracy: 0.4074
Epoch 147/2000
3/3 [==============================] - 1s 228ms/step - loss: 1.1770 - categorical_accuracy: 0.3333
Epoch 148/2000
3/3 [==============================] - 1s 226ms/step - loss: 1.1116 - categorical_accuracy: 0.4321
Epoch 149/2000
3/3 [==============================] - 1s 237ms/step - loss: 1.1461 - categorical_accuracy: 0.4815
Epoch 150/2000
3/3 [==============================] - 1s 233ms/step - loss: 1.0135 - categorical_accuracy: 0.3457
Epoch 151/2000
3/3 [==============================] - 1s 224ms/step - loss: 1.0778 - categorical_accuracy: 0.4074
Epoch 152/2000
3/3 [==============================] - 1s 245ms/step - loss: 1.4210 - categorical_accuracy: 0.3827
Epoch 153/2000
3/3 [==============================] - 1s 237ms/step - loss: 1.5006 - categorical_accura

3/3 [==============================] - 1s 210ms/step - loss: 1.0513 - categorical_accuracy: 0.4074
Epoch 217/2000
3/3 [==============================] - 1s 214ms/step - loss: 1.0714 - categorical_accuracy: 0.5062
Epoch 218/2000
3/3 [==============================] - 1s 217ms/step - loss: 1.2265 - categorical_accuracy: 0.4321
Epoch 219/2000
3/3 [==============================] - 1s 221ms/step - loss: 1.3384 - categorical_accuracy: 0.3827
Epoch 220/2000
3/3 [==============================] - 1s 222ms/step - loss: 1.1919 - categorical_accuracy: 0.4321
Epoch 221/2000
3/3 [==============================] - 1s 220ms/step - loss: 1.0126 - categorical_accuracy: 0.4321
Epoch 222/2000
3/3 [==============================] - 1s 229ms/step - loss: 1.0152 - categorical_accuracy: 0.4074
Epoch 223/2000
3/3 [==============================] - 1s 222ms/step - loss: 1.0172 - categorical_accuracy: 0.6543
Epoch 224/2000
3/3 [==============================] - 1s 220ms/step - loss: 1.1051 - categorical_accura

KeyboardInterrupt: 

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

# Make Prediction

In [36]:
#load signs, x_test, y_test
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))

In [37]:
res = model.predict(x_test)

In [38]:
signs

array(['Hello', 'Food', 'Washroom'], dtype='<U8')

In [39]:
signs[np.argmax(res[5])]

'Hello'

In [40]:
signs[np.argmax(y_test[5])]

'Hello'

In [40]:
res.shape

(9, 3)

In [41]:
y_test

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [42]:
res

array([[5.3766727e-02, 2.4770682e-01, 6.9852644e-01],
       [8.0655706e-01, 2.0600162e-02, 1.7284285e-01],
       [8.0121195e-01, 2.1262284e-02, 1.7752577e-01],
       [1.4012504e-06, 8.5017389e-01, 1.4982472e-01],
       [1.4011457e-06, 8.4682757e-01, 1.5317097e-01],
       [1.7091527e-06, 8.4000665e-01, 1.5999164e-01],
       [2.0998116e-06, 8.3362406e-01, 1.6637386e-01],
       [5.3696368e-07, 8.7554127e-01, 1.2445823e-01],
       [1.0858611e-06, 8.5759461e-01, 1.4240421e-01]], dtype=float32)

In [43]:
np.argmax(res[0])

2

In [44]:
np.argmax(y_test[0])

2

# Save Model

In [41]:
model.save('SLR.h5')

# Load Model

In [65]:
model=createModel()

In [66]:
model.load_weights('SLR.h5')

# Evaluation using confusion matrix

In [42]:
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix, accuracy_score

In [43]:
true_labels=[]
for label in y_test:
    index=np.argmax(label)
    true_labels.append(signs[index])

In [44]:
true_labels

['Washroom',
 'Washroom',
 'Washroom',
 'Hello',
 'Food',
 'Hello',
 'Food',
 'Food',
 'Hello']

In [45]:
y_predicted = model.predict(x_test)

In [46]:
predicted_labels=[]
for label in y_predicted:
    index=np.argmax(label)
    predicted_labels.append(signs[index])

In [47]:
predicted_labels

['Washroom',
 'Washroom',
 'Washroom',
 'Hello',
 'Food',
 'Hello',
 'Food',
 'Food',
 'Hello']

In [48]:
y_true = np.argmax(y_test, axis=1).tolist()

In [49]:
y_predicted=np.argmax(y_predicted,axis=1).tolist()

In [50]:
y_true

[2, 2, 2, 0, 1, 0, 1, 1, 0]

In [51]:
y_test

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [52]:
y_test.shape

(9, 3)

In [53]:
y_predicted

[2, 2, 2, 0, 1, 0, 1, 1, 0]

In [54]:
signs

array(['Hello', 'Food', 'Washroom'], dtype='<U8')

In [55]:
multilabel_confusion_matrix(true_labels, predicted_labels,labels=signs)

array([[[6, 0],
        [0, 3]],

       [[6, 0],
        [0, 3]],

       [[6, 0],
        [0, 3]]], dtype=int64)

In [56]:
confusion_matrix(y_true, y_predicted)

array([[3, 0, 0],
       [0, 3, 0],
       [0, 0, 3]], dtype=int64)

In [57]:
accuracy_score(y_true,y_predicted)

1.0

In [85]:
y_test.tolist()

[[0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0]]

In [100]:
list(zip(true_labels, predicted_labels))

[('Food', 'Food'),
 ('Food', 'Food'),
 ('Hungry', 'Hungry'),
 ('Thank You', 'Thank You'),
 ('Thank You', 'Thank You'),
 ('Thank You', 'Thank You'),
 ('Hello', 'Hello'),
 ('Food', 'Hungry'),
 ('Hungry', 'Hungry'),
 ('Hungry', 'Hungry'),
 ('Hungry', 'Hungry'),
 ('Food', 'Food')]